## Time 15

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from project_lib import Project
project = Project(sc, '0b89d6cb-bbe8-4e28-b211-c123c318d187', 'p-0238cf25b9be2b55b2c1531b8f35d3d44725ed84')
pc = project.project_context




credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-f8644b24-f3a5-402d-ad9c-def58df70dca',
    'IBM_API_KEY_ID': '0j9-70MzuGCxsqPXVeE2H2F27-L8ptcRo-0oMn4Mr05W',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'hackathon-donotdelete-pr-xe1bajv3coopx1',
    'FILE': 'olist_customers_dataset.csv'
}

from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200802184635-0008
KERNEL_ID = 21310520-e5ca-447c-9cbf-f9bf438934e1


## Funções 

In [2]:
# verifica colunas nulas
def check_null_col(df):
    qtd_linhas = df.count()
    colunas = df.columns
    for c in colunas:
        desc = df.describe()
        qtd_linhas_col = int(desc.collect()[0][1])
        if qtd_linhas > qtd_linhas_col:
            return c

# verifica linhas duplicadas
def check_line_duplic(df):
    qtd_linhas = df.count()
    qtd_linhas_distinct = df.distinct().count()
    if qtd_linhas > qtd_linhas_distinct:
        return df.dropDuplicates()
    else:
        return df

## Data Prep

In [3]:
## IMPORTANDO DATASETs

# dataset informacoes vendedor
cos.download_file(Bucket=credentials['BUCKET'],Key='olist_sellers_dataset.csv',Filename='olist_sellers_dataset.csv')
df_vendedor = spark.read.option('header','true').csv('olist_sellers_dataset.csv')

# dataset informacoes geolocalizacao
cos.download_file(Bucket=credentials['BUCKET'],Key='olist_geolocation_dataset.csv',Filename='olist_geolocation_dataset.csv')
df_geolocal = spark.read.option('header','true').csv('olist_geolocation_dataset.csv')

# dataset informacoes item venda
cos.download_file(Bucket=credentials['BUCKET'],Key='olist_order_items_dataset.csv',Filename='olist_order_items_dataset.csv')
df_item_pedido = spark.read.option('header','true').csv('olist_order_items_dataset.csv')

# dataset informacoes pagamento
cos.download_file(Bucket=credentials['BUCKET'],Key='olist_order_payments_dataset.csv',Filename='olist_order_payments_dataset.csv')
df_pagamento = spark.read.option('header','true').csv('olist_order_payments_dataset.csv')

# dataset informacoes item avaliacao
cos.download_file(Bucket=credentials['BUCKET'],Key='olist_order_reviews_dataset.csv',Filename='olist_order_reviews_dataset.csv')
df_avaliacao = spark.read.option('header','true').csv('olist_order_reviews_dataset.csv')

# dataset informacoes pedido
cos.download_file(Bucket=credentials['BUCKET'],Key='olist_orders_dataset.csv',Filename='olist_orders_dataset.csv')
df_pedido = spark.read.option('header','true').csv('olist_orders_dataset.csv')

# dataset informacoes produto
cos.download_file(Bucket=credentials['BUCKET'],Key='olist_products_dataset.csv',Filename='olist_products_dataset.csv')
df_produto = spark.read.option('header','true').csv('olist_products_dataset.csv')

# dataset informacoes cliente
cos.download_file(Bucket=credentials['BUCKET'],Key='olist_customers_dataset.csv',Filename='olist_customers_dataset.csv')
df_cliente = spark.read.option('header','true').csv('olist_customers_dataset.csv')

# dataset informacoes depara nome produto
cos.download_file(Bucket=credentials['BUCKET'],Key='product_category_name_translation.csv',Filename='product_category_name_translation.csv')
df_depara_prod = spark.read.option('header','true').csv('product_category_name_translation.csv')


In [4]:
## VERIFICANDO DUPLICIDADES

df_vendedor = check_line_duplic(df_vendedor)
df_geolocal = check_line_duplic(df_geolocal)
df_item_pedido = check_line_duplic(df_item_pedido)
df_pagamento = check_line_duplic(df_pagamento)
df_avaliacao = check_line_duplic(df_avaliacao)
df_pedido = check_line_duplic(df_pedido)
df_produto = check_line_duplic(df_produto)
df_cliente = check_line_duplic(df_cliente)
df_depara_prod = check_line_duplic(df_depara_prod)



In [5]:
## VERIFICANDO CAMPOS NULOS

# vendedor
check_null_col(df_vendedor)

In [6]:
# geolocalizacao
check_null_col(df_geolocal)

In [7]:
# pedido
check_null_col(df_item_pedido)

In [8]:
# pagamento
check_null_col(df_pagamento)

In [9]:
# avaliacao
check_null_col(df_avaliacao)

'review_id'

In [10]:
df_avaliacao.filter(df_avaliacao.review_id.isNull()).show()

+---------+--------------------+--------------------+--------------------+----------------------+--------------------+-----------------------+
|review_id|            order_id|        review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---------+--------------------+--------------------+--------------------+----------------------+--------------------+-----------------------+
|     null|material de boa q...|pensei que era ma...|mas e muito bonit...|   2018-01-06 00:00:00| 2018-01-08 14:20:31|                   null|
+---------+--------------------+--------------------+--------------------+----------------------+--------------------+-----------------------+



In [11]:
df_avaliacao = df_avaliacao.filter(df_avaliacao.review_id.isNotNull())

In [12]:
# pedido
check_null_col(df_pedido)

In [13]:
# produto
check_null_col(df_produto)

In [14]:
# cliente
check_null_col(df_cliente)

In [15]:
# depara produto

check_null_col(df_depara_prod)

In [16]:
## VERIFICANDO TIPOS DE COLUNAS

# vendedor
df_vendedor.dtypes

[('seller_id', 'string'),
 ('seller_zip_code_prefix', 'string'),
 ('seller_city', 'string'),
 ('seller_state', 'string')]

In [17]:
df_vendedor.show()

+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                 04195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
|c240c4061717ac180...|                 20920|   rio de janeiro|          RJ|
|e49c26c3edfa46d22...|                 55325|           brejao|          PE|
|1b938a7ec6ac5061a...|                 16304|        penapolis|          SP|
|768a86e36ad6aae3d...|                 01529|        sao paulo|          SP|
|ccc4bbb5f32a6ab2b...|                 80310|         curitiba|          PR|

In [18]:
# geolocalizacao
df_geolocal.dtypes

[('geolocation_zip_code_prefix', 'string'),
 ('geolocation_lat', 'string'),
 ('geolocation_lng', 'string'),
 ('geolocation_city', 'string'),
 ('geolocation_state', 'string')]

In [19]:
df_geolocal.show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                      01006| -23.54909870413756| -46.63584104801203|       sao paulo|               SP|
|                      01033| -23.53784452510254|-46.634192025535214|       sao paulo|               SP|
|                      01130| -23.52372003497028| -46.64531766541634|       sao paulo|               SP|
|                      01222|-23.545460503750906| -46.64710252553522|       sao paulo|               SP|
|                      01210|-23.538204848462094| -46.63944608742571|       sao paulo|               SP|
|                      01233|-23.532868152374615| -46.66033569525076|       são paulo|               SP|
|                      01201|-23.534308277829044| -46.6

In [20]:
# pagamento
df_pagamento.dtypes

[('order_id', 'string'),
 ('payment_sequential', 'string'),
 ('payment_type', 'string'),
 ('payment_installments', 'string'),
 ('payment_value', 'string')]

In [21]:
df_pagamento.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [22]:
df_pagamento = df_pagamento.select('order_id','payment_sequential','payment_type',\
                      df_pagamento.payment_installments.cast('int'),\
                      df_pagamento.payment_value.cast('float')\
                     )

In [23]:
# item pedido
df_item_pedido.dtypes

[('order_id', 'string'),
 ('order_item_id', 'string'),
 ('product_id', 'string'),
 ('seller_id', 'string'),
 ('shipping_limit_date', 'string'),
 ('price', 'string'),
 ('freight_value', 'string')]

In [24]:
df_item_pedido.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.90|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.90|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.00|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.90|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [25]:
df_item_pedido = df_item_pedido.select('order_id','order_item_id','product_id','seller_id',\
                      df_item_pedido.shipping_limit_date.cast('timestamp'),\
                      df_item_pedido.price.cast('float'),\
                      df_item_pedido.freight_value.cast('float')
                     )

In [26]:
# avaliacao
df_avaliacao.dtypes

[('review_id', 'string'),
 ('order_id', 'string'),
 ('review_score', 'string'),
 ('review_comment_title', 'string'),
 ('review_comment_message', 'string'),
 ('review_creation_date', 'string'),
 ('review_answer_timestamp', 'string')]

In [27]:
df_avaliacao.show()

+--------------------+--------------------+-------------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|       review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+-------------------+--------------------+----------------------+--------------------+-----------------------+
|aecdc81e65fd3fb1f...|b70c2a4797319570d...|                  5|Sem dúvida recomendo|  Loja extremamente...| 2018-07-15 00:00:00|    2018-07-17 18:17:39|
|b7cf91f7c1000f7b9...|4122d084b6a345866...|                  3|                null|  o rastreio não te...| 2017-09-16 00:00:00|    2017-09-16 09:27:46|
|3786ebaae8f304e91...|2a2ef9160ab84c882...|                  5|                null|                  null| 2018-05-23 00:00:00|    2018-05-23 16:21:50|
|69bd9e3c7d45c7ec2...|517b4b7e7f723df36...|                  5|                nul

In [28]:
df_avaliacao = df_avaliacao.select('review_id','order_id',\
                                   df_avaliacao.review_score.cast('int'),\
                                   'review_comment_title','review_comment_message',\
                                   df_avaliacao.review_creation_date.cast('timestamp'),\
                                   df_avaliacao.review_answer_timestamp.cast('timestamp')
                                  )

In [29]:
# pedido
df_pedido.dtypes

[('order_id', 'string'),
 ('customer_id', 'string'),
 ('order_status', 'string'),
 ('order_purchase_timestamp', 'string'),
 ('order_approved_at', 'string'),
 ('order_delivered_carrier_date', 'string'),
 ('order_delivered_customer_date', 'string'),
 ('order_estimated_delivery_date', 'string')]

In [30]:
df_pedido.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [31]:
df_pedido = df_pedido.select('order_id','customer_id','order_status',\
                             df_pedido.order_purchase_timestamp.cast('timestamp'),\
                             df_pedido.order_approved_at.cast('timestamp'),\
                             df_pedido.order_delivered_carrier_date.cast('timestamp'),\
                             df_pedido.order_delivered_customer_date.cast('timestamp'),\
                             df_pedido.order_estimated_delivery_date.cast('timestamp')\
                            )

In [32]:
# pedido
df_produto.dtypes

[('product_id', 'string'),
 ('product_category_name', 'string'),
 ('product_name_lenght', 'string'),
 ('product_description_lenght', 'string'),
 ('product_photos_qty', 'string'),
 ('product_weight_g', 'string'),
 ('product_length_cm', 'string'),
 ('product_height_cm', 'string'),
 ('product_width_cm', 'string')]

In [33]:
df_produto.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [34]:
# cliente
df_cliente.dtypes

[('customer_id', 'string'),
 ('customer_unique_id', 'string'),
 ('customer_zip_code_prefix', 'string'),
 ('customer_city', 'string'),
 ('customer_state', 'string')]

In [35]:
df_cliente.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                   09790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                   01151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                   08775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [36]:
# depara_produto
df_depara_prod.dtypes

[('product_category_name', 'string'),
 ('product_category_name_english', 'string')]

In [37]:
df_depara_prod.show()

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|         beleza_saude|                health_beauty|
| informatica_acess...|         computers_accesso...|
|           automotivo|                         auto|
|      cama_mesa_banho|               bed_bath_table|
|     moveis_decoracao|              furniture_decor|
|        esporte_lazer|               sports_leisure|
|           perfumaria|                    perfumery|
| utilidades_domest...|                   housewares|
|            telefonia|                    telephony|
|   relogios_presentes|                watches_gifts|
|    alimentos_bebidas|                   food_drink|
|                bebes|                         baby|
|            papelaria|                   stationery|
| tablets_impressao...|         tablets_printing_...|
|           brinquedos|                         toys|
|       telefonia_fixa|     

In [38]:
df_depara_prod.show(df_depara_prod.count())

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|         beleza_saude|                health_beauty|
| informatica_acess...|         computers_accesso...|
|           automotivo|                         auto|
|      cama_mesa_banho|               bed_bath_table|
|     moveis_decoracao|              furniture_decor|
|        esporte_lazer|               sports_leisure|
|           perfumaria|                    perfumery|
| utilidades_domest...|                   housewares|
|            telefonia|                    telephony|
|   relogios_presentes|                watches_gifts|
|    alimentos_bebidas|                   food_drink|
|                bebes|                         baby|
|            papelaria|                   stationery|
| tablets_impressao...|         tablets_printing_...|
|           brinquedos|                         toys|
|       telefonia_fixa|     

## Transformacoes

In [39]:
##  quantidade de vendas por categoria de produtos por estado
df_pedido_estado = df_pedido.join(df_cliente, df_pedido.customer_id == df_cliente.customer_id)\
.drop(df_cliente.customer_id)

df_podutos_pedido = df_item_pedido.join(df_produto, df_item_pedido.product_id == df_produto.product_id)\
.drop(df_produto.product_id)\
.groupBy('order_id','product_category_name').count()

df_pedido_prod_estado = df_pedido_estado.join(df_podutos_pedido, df_pedido_estado.order_id == df_podutos_pedido.order_id)\
.drop(df_podutos_pedido.order_id)\
.groupBy('customer_state','product_category_name').sum('count')\
.withColumnRenamed('sum(count)','qtd_produto')

df_qtd_vend_prod_estado = df_pedido_prod_estado.orderBy(df_pedido_prod_estado.customer_state,df_pedido_prod_estado.qtd_produto.desc())







## Estudos

In [40]:
# quanditade de vendas por categoria de produtos (top 3) por estado

df_1 = df_qtd_vend_prod_estado.withColumn('rk', F.row_number().over(Window.partitionBy('customer_state')\
                                                             .orderBy(df_qtd_vend_prod_estado.qtd_produto.desc())))

df_1.filter(df_1.rk <= 3).show()



+--------------+---------------------+-----------+---+
|customer_state|product_category_name|qtd_produto| rk|
+--------------+---------------------+-----------+---+
|            SC|        esporte_lazer|        363|  1|
|            SC|     moveis_decoracao|        354|  2|
|            SC|      cama_mesa_banho|        330|  3|
|            RO|         beleza_saude|         25|  1|
|            RO| informatica_acess...|         23|  2|
|            RO|        esporte_lazer|         20|  3|
|            PI|         beleza_saude|         54|  1|
|            PI|   relogios_presentes|         41|  2|
|            PI|            telefonia|         38|  3|
|            AM|         beleza_saude|         20|  1|
|            AM| informatica_acess...|         17|  2|
|            AM|            telefonia|         15|  3|
|            RR|        esporte_lazer|          8|  1|
|            RR|         beleza_saude|          7|  2|
|            RR|     moveis_decoracao|          6|  3|
|         

#TESTES





from project_lib import Project
project = Project(sc, '0b89d6cb-bbe8-4e28-b211-c123c318d187', 'p-0238cf25b9be2b55b2c1531b8f35d3d44725ed84')
pc = project.project_context

pandas_df = df_1.toPandas()
project.save_data("teste.csv", pandas_df.to_csv(index=False))

In [48]:
## dados do tableau MVP

# join pedido com item pedido
df_podutos_pedido_mvp = df_item_pedido.join(df_produto, df_item_pedido.product_id == df_produto.product_id)\
.drop(df_produto.product_id)


112650

In [49]:
# filtra somente o segmento da loja mvp
df_podutos_pedido_mvp = df_podutos_pedido_mvp.filter((df_podutos_pedido_mvp.product_category_name == 'cama_mesa_banho')\
                                                     | (df_podutos_pedido_mvp.product_category_name == 'beleza_saude')\
                                                     | (df_podutos_pedido_mvp.product_category_name == 'esporte_lazer')
                                                    )
df_podutos_pedido_mvp.count()

29426

In [53]:

# join com endereco do comprador
df_pedido_estado_mvp = df_podutos_pedido_mvp.join(df_pedido_estado, df_podutos_pedido_mvp.order_id == df_pedido_estado.order_id)\
.drop(df_pedido_estado.order_id)

df_pedido_estado_mvp.count()

df_pedido_estado_mvp.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+------------------------+--------------------+--------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|  customer_unique_id|custom

In [ ]:
# tatra df geolocal
df_geolocal_mvp = df_geolocal.withColumn('rk', F.row_number().over(Window.partitionBy('geolocation_zip_code_prefix')\
                                                             .orderBy('geolocation_state')))
df_geolocal_mvp = df_geolocal_mvp.filter(df_geolocal_mvp.rk <= 1)

In [80]:
# join com info da geolocalizacao
df_pedido_geoloc_mpv = df_pedido_estado_mvp.join(df_geolocal_mvp, df_pedido_estado_mvp.customer_zip_code_prefix == df_geolocal.geolocation_zip_code_prefix)

df_pedido_geoloc_mpv.count()

29352

In [81]:
pandas_df = df_pedido_geoloc_mpv.toPandas()
project.save_data("dados_tableau.csv", pandas_df.to_csv(index=False))

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,...,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,rk
0,b7d6b37701289908bb8ae3a30fca73bb,1,fef63ef09494166bda2812ff585be979,e9779976487b77c6d4ac45f75ec7afe9,2017-06-01 11:15:11,58.000000,8.270000,esporte_lazer,57,3119,...,9c28a565adee8c7bc1852b1278fb18c4,02053,sao paulo,SP,02053,-23.520545905360922,-46.607006766576305,sao paulo,SP,1
1,7ab737441b79ec93e9e9b1335f22a30c,1,0152f69b6cf919bcdaf117aa8c43e5a2,d2374cbcbb3ca4ab1086534108cc3ab7,2018-03-26 03:50:42,13.900000,11.850000,cama_mesa_banho,40,116,...,65a3a1c5c86631736e9ad00c3eaec686,02053,sao paulo,SP,02053,-23.520545905360922,-46.607006766576305,sao paulo,SP,1
2,3168875baaa7b1b7ba3e833c53b34fe0,1,1f30aacc69904c0d0bf8d05cb2abef72,4a3ca9315b744ce9f8e9374361493884,2018-04-03 18:08:46,135.000000,16.610001,cama_mesa_banho,58,293,...,c696a6c5d023c3e981b88dcf52b34fd5,02053,sao paulo,SP,02053,-23.520545905360922,-46.607006766576305,sao paulo,SP,1
3,791a044148a5de3317595143ddcbd844,1,e729528d0513c6dfec158a515a47187b,955fee9216a65b617aa5c0531780ce60,2018-01-18 11:08:59,59.900002,13.080000,esporte_lazer,56,574,...,efafdbc8cc9e64a4fdaa7c3089e3e2de,02943,sao paulo,SP,02943,-23.477696968636398,-46.7247580037509,sao paulo,SP,1
4,989efe8965cc3848a0358e9445c743ef,1,625522e2041f5759bc483af612e22718,951e8cef368f09bb3f3d03c00ca4702c,2018-05-16 03:09:52,58.990002,17.320000,cama_mesa_banho,41,582,...,82ba4e9e9e88636659d19fe022ef7498,02943,sao paulo,SP,02943,-23.477696968636398,-46.7247580037509,sao paulo,SP,1
5,7282809524b9b429bb37460b46596293,1,5bd7e57735a44e529bf066fee83a9c60,609e1a9a6c2539919b8205cf7c4e6ff0,2018-06-29 12:15:29,59.900002,19.610001,cama_mesa_banho,56,684,...,0068bd2e9e76c018846dbc6e2bec5ac5,03904,sao paulo,SP,03904,-23.57813561726011,-46.51637902400601,são paulo,SP,1
6,fb7b22bb2a29860909a25f91510e4292,1,372645c7439f9661fbbacfd129aa92ec,da8622b14eb17ae2831f4ac5b9dab84a,2018-08-29 02:45:19,114.900002,13.300000,cama_mesa_banho,60,236,...,72ff384d757b9a86b989214e8801b21f,04138,sao paulo,SP,04138,-23.613086357204672,-46.63628029808413,sao paulo,SP,1
7,fb7b22bb2a29860909a25f91510e4292,2,120fa011365fc39efe382cba4e50999e,da8622b14eb17ae2831f4ac5b9dab84a,2018-08-29 02:45:19,179.899994,10.590000,cama_mesa_banho,60,234,...,72ff384d757b9a86b989214e8801b21f,04138,sao paulo,SP,04138,-23.613086357204672,-46.63628029808413,sao paulo,SP,1
8,fb7b22bb2a29860909a25f91510e4292,3,372645c7439f9661fbbacfd129aa92ec,da8622b14eb17ae2831f4ac5b9dab84a,2018-08-29 02:45:19,114.900002,13.300000,cama_mesa_banho,60,236,...,72ff384d757b9a86b989214e8801b21f,04138,sao paulo,SP,04138,-23.613086357204672,-46.63628029808413,sao paulo,SP,1
9,0c3190b4838c596fd1d6cd74e0436120,1,10d2b7a176cd8a35bcae7c15e0e7626e,8d956fec2e4337affcb520f56fd8cbfd,2018-03-07 08:15:27,68.989998,8.720000,esporte_lazer,58,1033,...,ccfd020eaf030301df959b4f09ff45b6,04438,sao paulo,SP,04438,-23.680380973317483,-46.67088252608841,são paulo,SP,1


In [82]:
project.save_data("dados_tableau.csv", pandas_df.to_csv(index=False))

{'file_name': 'dados_tableau.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'hackathon-donotdelete-pr-xe1bajv3coopx1',
 'asset_id': '9daf7cb5-01f2-4350-b9ca-fb400b8716b4'}